In [ ]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from acoustic_BEM.geometry import Body, Field
from acoustic_BEM.mesh import Mesh
from acoustic_BEM.integrators import ElementIntegratorCollocation
from acoustic_BEM.matrix_assembly import CollocationAssembler
from acoustic_BEM.solve import BEMSolver

%load_ext autoreload
%autoreload 2

In [ ]:
def icosahedron():
    t = (1.0 + np.sqrt(5.0)) / 2.0
    V = np.array([
        [-1,  t, 0],
        [ 1,  t, 0],
        [-1, -t, 0],
        [ 1, -t, 0],
        [ 0, -1,  t],
        [ 0,  1,  t],
        [ 0, -1, -t],
        [ 0,  1, -t],
        [ t,  0, -1],
        [ t,  0,  1],
        [-t,  0, -1],
        [-t,  0,  1],
    ], dtype=float)
    V /= np.linalg.norm(V, axis=1)[:, None]
    F = np.array([
        [0, 11, 5], [0, 5, 1], [0, 1, 7], [0, 7,10], [0,10,11],
        [1, 5, 9], [5,11, 4], [11,10,2], [10,7,6], [7, 1, 8],
        [3, 9, 4], [3, 4, 2], [3, 2, 6], [3, 6, 8], [3, 8, 9],
        [4, 9, 5], [2, 4,11], [6, 2,10], [8, 6, 7], [9, 8, 1]
    ], dtype=int)
    return V, F

def subdivide_sphere(V, F, levels=1):
    """Loop-subdivide each triangle and re-project to unit sphere."""
    for _ in range(levels):
        mid_cache = {}
        newF = []
        def midpoint(i, j):
            key = tuple(sorted((i, j)))
            if key in mid_cache: return mid_cache[key]
            m = (V[i] + V[j]) * 0.5
            m = m / np.linalg.norm(m)
            idx = len(V_list)
            V_list.append(m)
            mid_cache[key] = idx
            return idx
        V_list = [v.copy() for v in V]
        for a,b,c in F:
            ab = midpoint(a,b)
            bc = midpoint(b,c)
            ca = midpoint(c,a)
            newF += [
                [a, ab, ca],
                [b, bc, ab],
                [c, ca, bc],
                [ab, bc, ca],
            ]
        V = np.asarray(V_list)
        F = np.asarray(newF, dtype=int)
    return V, F

### Uncomment to visualize the mesh
a = 0.075              # sphere radius [m]
subdiv = 4
V, F = icosahedron()
V, F = subdivide_sphere(V, F, levels=subdiv)
V = a * V

pl = pv.Plotter()
pv_mesh = pv.PolyData(V, np.hstack([np.full((F.shape[0],1),3), F]))
pl.add_mesh(pv_mesh, show_edges=True)
pl.show()

In [ ]:
def analytic_phi_pulsating_sphere(r: np.ndarray, 
                                  a: float, 
                                  k: float, 
                                  vr: complex) -> np.ndarray:
    
    A = a/r * 1.225 * 343 * vr * (1j * k * a) / (1-1j*k*a)
    return A * np.exp(1j*k*(r - a))

In [ ]:
frequency = 54.59                         # frequency [Hz]
c0 = 343                               # speed of sound [m/s]
omega = 2 * np.pi * frequency          # angular frequency [rad/s]
k = omega / c0                         # wavenumber [rad/m]
rho0 = 1.225                           # density of air [kg/m^3]

radius = 1                         # radius of pulsating sphere [m]
Vr = 1                          # normal velocity on sphere surface [m/s]

plane_extent = 10
rs = np.linspace(0.5, plane_extent, 60)
p_ref = analytic_phi_pulsating_sphere(rs, radius, k, Vr)

plt.plot(rs/radius, 20 * np.log10(np.abs(p_ref)/(2 * 10**-5)), label='Analytic', marker='o', linestyle='--')

In [ ]:
frequency = np.linspace(0.001, 500, 100)  # frequency [Hz]
c0 = 343                               # speed of sound [m/s]
omega = 2 * np.pi * frequency          # angular frequency [rad/s]
k = omega / c0                         # wavenumber [rad/m]
rho0 = 1.225                           # density of air [kg/m^3]

radius = 1                         # radius of pulsating sphere [m]
Vr = 1                          # normal velocity on sphere surface [m/s]

plane_extent = 10
rs = 2
p_ref = analytic_phi_pulsating_sphere(rs, radius, k, Vr)

fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax1.plot(frequency, np.abs(p_ref), label='Analytic', marker='o', linestyle='--')
ax1.set_xlim(0, np.max(frequency))

ax2.set_xticks(np.linspace(0, 1, 9))
ax2.set_xticklabels([f"{tick:.1f}" for tick in np.linspace(0, np.max(k), 9)])
ax1.set_xlabel('Frequency [Hz]')
ax2.set_xlabel('Wavenumber [rad/m]')
ax1.set_ylabel('Pressure Magnitude [Pa]')
ax1.set_title('Pressure at r=2m from Pulsating Sphere')

In [ ]:
subdiv = 2                          # mesh subdivision level
nodes, elements = icosahedron()
nodes, elements = subdivide_sphere(nodes, elements, levels=subdiv)
nodes *= radius

vel_BC = np.ones(nodes.shape[0]) * Vr

field_pts_safe = np.array([0, 0, 1])[:, None] * np.linspace(1.0, 3.0, 3)
field_pts_safe = field_pts_safe.T
field_pts_safe.shape

# print(f'Mesh has {nodes.shape[0]} vertices and {elements.shape[0]} faces.')
# print(f'Characteristic length: {char_length:.2f} m')
# print(f'Wavenumber: {k:.2f} 1/m')
# print(f'k*L = {k*char_length:.2f}')
# print(f'Wavelength: {2*np.pi/k:.2f} m')
# print(f'Velocity BC: {np.min(np.abs(vel_BC)):.4f} to {np.max(np.abs(vel_BC)):.4f} m/s')

In [ ]:
field_pts_safe

In [ ]:
elements.shape

In [ ]:
p_bem = []
p_bem_bm = []

sphere = Body(mesh_nodes=nodes, 
            mesh_elements=elements, 
            Neumann_BC=vel_BC,
            Dirichlet_BC=None,
            frequency=50)

field = Field(field_extent=np.array([[3, 3.5],
            [0, 0.5],
            [0, 0.5]]),
            num_points=np.array([1, 1, 1]),
            rho0=rho0,
            c0=c0,)

mesh = Mesh(source_object=sphere,
            peripheral_objects=None,
            field=field)

for f_i in tqdm(np.linspace(50, 500, 50)):
    mesh.k = 2 * np.pi * f_i / c0
    k_i = mesh.k
    integrator = ElementIntegratorCollocation(k = k_i,
                                            kernel_mode = "free")
    assembler = CollocationAssembler(mesh, integrator, quad_order=7)
    solver = BEMSolver(assembler=assembler)
    mats = {
        "S": assembler.assemble("S", verbose=False),
        "D": assembler.assemble("D", verbose=False),
    }
    # mats_bm = {
    #     "S": mats["S"],
    #     "D": mats["D"],
    #     "Kp": assembler.assemble("Kp", verbose=False),
    #     "N": assembler.assemble("NReg", verbose=False),
    # }

    omega = 2 * np.pi * f_i

    phi_bnd = solver.solve_direct(matrices=mats)

    phi_field = solver.evaluate_field(field_pts_safe, quad_order=7, verbose=False)
    p_field = - 1j * omega * rho0 * phi_field
    p_bem.append(p_field)

    
    # phi_bnd = solver.solve_burton_miller(matrices=mats_bm,
    #                                      alpha = 1j/k_i)
    
    # phi_field = solver.evaluate_field(field_pts_safe, quad_order=7, verbose=False)
    # p_field = - 1j * omega * rho0 * phi_field
    # p_bem_bm.append(p_field)

In [ ]:
fig = plt.figure(figsize=(14,6), tight_layout=True)
ax1 = fig.add_subplot(131)

ax1.plot(frequency, np.abs(p_ref), label='Analytic')
ax1.plot(np.linspace(50, 500, 50), np.abs(p_bem), label='BEM', linestyle='--')
# ax1.plot(np.linspace(50, 500, 50), np.abs(p_bem_bm), label='BEM BM', linestyle='--')
ax1.axvline(x = 3.14 * c0 / (2 * np.pi), color='gray', linestyle=':')
ax1.axvline(x = 6.28 * c0 / (2 * np.pi), color='gray', linestyle=':')

ax12 = ax1.twiny()
ax1.set_xlim(0, np.max(frequency))
ax12.set_xticks(np.linspace(0, 1, 9))
ax12.set_xticklabels([f"{tick:.1f}" for tick in np.linspace(0, np.max(k), 9)])
ax12.set_xlabel('Wavenumber [rad/m]')

ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel('Sound Pressure [Pa]')

ax2 = fig.add_subplot(132)

ax2.plot(frequency, np.real(p_ref), label='Analytic')
ax2.plot(np.linspace(50, 500, 50), np.real(p_bem), label='BEM', linestyle='--')
# ax2.plot(np.linspace(50, 500, 50), np.real(p_bem_bm), label='BEM BM', linestyle='--')

ax22 = ax2.twiny()
ax2.set_xlim(0, np.max(frequency))
ax22.set_xticks(np.linspace(0, 1, 9))
ax22.set_xticklabels([f"{tick:.1f}" for tick in np.linspace(0, np.max(k), 9)])
ax22.set_xlabel('Wavenumber [rad/m]')

ax2.set_xlabel('Frequency [Hz]')
ax2.set_ylabel('Sound Pressure Real [Pa]')

ax3 = fig.add_subplot(133)

ax3.plot(frequency, np.imag(p_ref), label='Analytic')
ax3.plot(np.linspace(50, 500, 50), np.imag(p_bem), label='BEM', linestyle='--')
# ax3.plot(np.linspace(50, 500, 50), np.imag(p_bem_bm), label='BEM BM', linestyle='--')

ax32 = ax3.twiny()
ax3.set_xlim(0, np.max(frequency))
ax32.set_xticks(np.linspace(0, 1, 9))
ax32.set_xticklabels([f"{tick:.1f}" for tick in np.linspace(0, np.max(k), 9)])
ax32.set_xlabel('Wavenumber [rad/m]')

ax3.set_xlabel('Frequency [Hz]')
ax3.set_ylabel('Sound Pressure Imag [Pa]')

ax3.legend(loc = (1.01, 0.8))
plt.suptitle('Pressure at r=2m from Pulsating Sphere')